In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("labeled_sensor_data.csv")

# Check for missing values in x, y, and z columns
missing_values = df[['x', 'y', 'z']].isnull().sum()

print("Missing values in each column:")
print(missing_values)


In [ ]:
duplicate_rows = df[df.duplicated()]

print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")
print("Duplicate rows (if any):")
print(duplicate_rows)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Standardize x, y, z columns
df[['x', 'y', 'z']] = scaler.fit_transform(df[['x', 'y', 'z']])

In [ ]:
# Sort by subject, activity_type, and timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed', dayfirst=False, errors='coerce').dt.tz_convert('UTC')
df = df.sort_values(['subject', 'activity_type', 'timestamp'])

# Compute time differences within each subject and activity_type group
df['time_diff'] = df.groupby(['subject', 'activity_type'])['timestamp'].diff().dt.total_seconds()

# Assign a new sequence_id when time_diff > 1 second (indicating a gap)
df['sequence_id'] = (df['time_diff'] > 1.0).cumsum()

# Drop time_diff as it’s no longer needed
df = df.drop(columns=['time_diff'])

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

def generate_windows(group, window_size=pd.Timedelta(seconds=2), step_size=pd.Timedelta(seconds=1)):
    group = group.sort_values('timestamp')
    start_time = group['timestamp'].min()
    end_time = group['timestamp'].max()
    windows = []
    labels = []
    current_start = start_time
    
    while current_start + window_size <= end_time:
        window_end = current_start + window_size
        window_data = group[(group['timestamp'] >= current_start) & (group['timestamp'] < window_end)]
        if not window_data.empty:
            windows.append(window_data)
            labels.append(group['activity_type'].iloc[0])  # All rows have the same activity_type
        current_start += step_size
    
    return windows, labels

# Process all sequences
all_windows = []
all_labels = []
for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
    windows, labels = generate_windows(group)
    all_windows.extend(windows)
    all_labels.extend(labels)

In [ ]:
def extract_features(window_data):
    features = {}
    
    # Add subject
    features['subject'] = window_data['subject'].iloc[0]
    
    # Time-Domain Features
    for axis in ['x', 'y', 'z']:
        data = window_data[axis]
        features[f'{axis}_mean'] = data.mean()
        features[f'{axis}_std'] = data.std()
        features[f'{axis}_skew'] = skew(data)
        features[f'{axis}_kurt'] = kurtosis(data)
    
    # Magnitude Features
    mag = np.sqrt(window_data['x']**2 + window_data['y']**2 + window_data['z']**2)
    features['mag_mean'] = mag.mean()
    features['mag_std'] = mag.std()
    
    # Frequency-Domain Features
    for axis in ['x', 'y', 'z']:
        fft_vals = np.abs(np.fft.fft(window_data[axis]))[:len(window_data[axis])//2]
        freqs = np.fft.fftfreq(len(window_data[axis]), d=0.135)[:len(window_data[axis])//2]
        for low, high in [(0, 1), (1, 2), (2, 3), (3, 3.7)]:
            mask = (freqs >= low) & (freqs < high)
            features[f'{axis}_power_{low}-{high}'] = np.sum(fft_vals[mask]**2) if mask.any() else 0
    
    # Step Frequency
    peaks, _ = find_peaks(mag, height=mag.mean() + mag.std(), distance=3)
    window_duration = (window_data['timestamp'].max() - window_data['timestamp'].min()).total_seconds()
    features['step_frequency'] = len(peaks) / window_duration if window_duration > 0 else 0
    
    return features


# Extract features from all windows
feature_list = [extract_features(window) for window in all_windows]
features_df = pd.DataFrame(feature_list)
features_df['label'] = all_labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
features_df['label'] = le.fit_transform(features_df['label'])

In [ ]:
subjects = features_df['subject'].unique()
train_subjects = subjects[:6]
test_subjects = subjects[6:]

train_df = features_df[features_df['subject'].isin(train_subjects)]
test_df = features_df[features_df['subject'].isin(test_subjects)]


X_train = train_df.drop(columns=['label', 'subject'])
y_train = train_df['label']
X_test = test_df.drop(columns=['label', 'subject'])
y_test = test_df['label']

In [ ]:
train_df.subject.value_counts(),train_subjects,test_subjects

In [ ]:
train_df

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter

# Data Preprocessing Functions
def generate_windows(group, window_size=2*50, step_size=1*50):  # 2s window, 1s step at 50Hz
    windows = []
    labels = []
    data = group[['x', 'y', 'z']].values
    activity = group['activity_type'].iloc[0]
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data[start:start + window_size]
        windows.append(window)
        labels.append(activity)
    return windows, labels

def extract_features(window):
    features = []
    for axis in range(3):  # x, y, z
        data = window[:, axis]
        features.extend([data.mean(), data.std(), skew(data), kurtosis(data)])
    mag = np.sqrt(window[:, 0]**2 + window[:, 1]**2 + window[:, 2]**2)
    features.extend([mag.mean(), mag.std()])
    return np.array(features)

# Prepare sequences and compute max sequence length
def prepare_sequences(df, max_seq_len=200):  # Adjustable max_seq_len
    sequences = []
    labels = []
    subjects = []
    max_len = 0
    for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
        windows, window_labels = generate_windows(group)
        if windows:
            sequence_features = [extract_features(window) for window in windows]
            seq_len = len(sequence_features)
            if seq_len > max_seq_len:
                sequence_features = sequence_features[:max_seq_len]  # Truncate if too long
                seq_len = max_seq_len
            max_len = max(max_len, seq_len)
            sequences.append(np.array(sequence_features))
            labels.append(window_labels[0])
            subjects.append(subject)
    print(f"Maximum sequence length: {max_len}")
    return sequences, labels, subjects, max_len

# Custom Dataset
class ActivityDataset(Dataset):
    def __init__(self, sequences, labels, subjects, subject_encoder):
        self.sequences = sequences
        self.labels = labels
        self.subjects = [subject_encoder.get(subj, len(subject_encoder)) for subj in subjects]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.subjects[idx]

def collate_fn(batch):
    sequences, labels, subjects = zip(*batch)
    max_len = max(seq.shape[0] for seq in sequences)
    padded_sequences = []
    masks = []
    for seq in sequences:
        pad_width = ((0, max_len - seq.shape[0]), (0, 0))
        padded_seq = np.pad(seq, pad_width, mode='constant', constant_values=0)
        padded_sequences.append(padded_seq)
        mask = np.ones(seq.shape[0], dtype=bool)
        mask = np.pad(mask, (0, max_len - seq.shape[0]), mode='constant', constant_values=False)
        masks.append(mask)
    return (torch.tensor(padded_sequences, dtype=torch.float32),
            torch.tensor(masks, dtype=torch.bool),
            torch.tensor(labels, dtype=torch.long),
            torch.tensor(subjects, dtype=torch.long))

# Transformer Model
class ActivityTransformer(nn.Module):
    def __init__(self, num_features, num_classes, num_subjects, max_seq_len, d_model=128, num_heads=4, num_layers=2, dropout=0.1, embedding_dim=32):
        super(ActivityTransformer, self).__init__()
        self.d_model = d_model
        self.subject_embedding = nn.Embedding(num_subjects + 1, embedding_dim)
        self.input_proj = nn.Linear(num_features + embedding_dim, d_model)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.pos_encoder = nn.Parameter(torch.zeros(1, max_seq_len + 1, d_model))  # +1 for CLS token
        encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads, dim_feedforward=512, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.classifier = nn.Linear(d_model, num_classes)
    
    def forward(self, x, mask, subject_ids):
        batch_size, seq_len, num_features = x.shape
        subj_emb = self.subject_embedding(subject_ids).unsqueeze(1).expand(-1, seq_len, -1)
        x = torch.cat((x, subj_emb), dim=-1)
        x = self.input_proj(x)
        cls_token = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        pos_encoding = self.pos_encoder[:, :seq_len + 1, :]  # Slice to match input length
        x += pos_encoding
        cls_mask = torch.ones(batch_size, 1, dtype=torch.bool, device=x.device)
        full_mask = torch.cat((cls_mask, mask), dim=1)
        transformer_mask = ~full_mask
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x, src_key_padding_mask=transformer_mask)
        x = x.permute(1, 0, 2)
        cls_output = x[:, 0, :]
        return self.classifier(cls_output)

# Training and Evaluation Functions
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        sequences, masks, labels, subjects = [b.to(device) for b in batch]
        optimizer.zero_grad()
        logits = model(sequences, masks, subjects)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            sequences, masks, labels, subjects = [b.to(device) for b in batch]
            logits = model(sequences, masks, subjects)
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    return accuracy_score(true, preds), f1_score(true, preds, average='weighted')

# Main Execution

    # Prepare data
sequences, labels, subjects, max_seq_len = prepare_sequences(df)
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
subject_encoder = {subj: idx for idx, subj in enumerate(sorted(set(subjects)))}

# Split for a single fold (example: leave out 'U3')
train_idx = [i for i, s in enumerate(subjects) if s != 'U3']
val_idx = [i for i, s in enumerate(subjects) if s == 'U3']
train_data = ([sequences[i] for i in train_idx], labels_encoded[train_idx], [subjects[i] for i in train_idx])
val_data = ([sequences[i] for i in val_idx], labels_encoded[val_idx], [subjects[i] for i in val_idx])

# Datasets and Loaders
train_dataset = ActivityDataset(*train_data, subject_encoder)
val_dataset = ActivityDataset(*val_data, subject_encoder)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Model Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = sequences[0].shape[1]
model = ActivityTransformer(
    num_features=num_features,
    num_classes=len(le.classes_),
    num_subjects=len(subject_encoder),
    max_seq_len=max_seq_len
).to(device)
class_weights = torch.tensor([1.0 / Counter(labels_encoded)[i] for i in range(len(le.classes_))], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Training Loop
for epoch in range(100):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_acc, val_f1 = evaluate(model, val_loader, device)
    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val F1={val_f1:.4f}")
    scheduler.step(train_loss)

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter

# Data Preprocessing Functions
def generate_windows(group, window_size=pd.Timedelta(seconds=2), step_size=pd.Timedelta(seconds=1)):
    group = group.sort_values('timestamp')
    start_time = group['timestamp'].min()
    end_time = group['timestamp'].max()
    windows = []
    labels = []
    subjects = []
    current_start = start_time
    
    while current_start + window_size <= end_time:
        window_end = current_start + window_size
        window_data = group[(group['timestamp'] >= current_start) & (group['timestamp'] < window_end)]
        if not window_data.empty:
            windows.append(window_data)
            labels.append(group['activity_type'].iloc[0])
            subjects.append(group['subject'].iloc[0])
        current_start += step_size
    
    return windows, labels, subjects

def extract_features(window_data):
    features = {}
    
    # Time-Domain Features
    for axis in ['x', 'y', 'z']:
        data = window_data[axis]
        features[f'{axis}_mean'] = data.mean()
        features[f'{axis}_std'] = data.std()
        features[f'{axis}_skew'] = skew(data)
        features[f'{axis}_kurt'] = kurtosis(data)
    
    # Magnitude Features
    mag = np.sqrt(window_data['x']**2 + window_data['y']**2 + window_data['z']**2)
    features['mag_mean'] = mag.mean()
    features['mag_std'] = mag.std()
    
    # Frequency-Domain Features
    # Assuming 7.4 Hz sampling rate (0.135s interval)
    for axis in ['x', 'y', 'z']:
        fft_vals = np.abs(np.fft.fft(window_data[axis]))[:len(window_data[axis])//2]
        freqs = np.fft.fftfreq(len(window_data[axis]), d=0.135)[:len(window_data[axis])//2]
        for low, high in [(0, 1), (1, 2), (2, 3), (3, 3.7)]:
            mask = (freqs >= low) & (freqs < high)
            features[f'{axis}_power_{low}-{high}'] = np.sum(fft_vals[mask]**2) if mask.any() else 0
    
    # Step Frequency
    peaks, _ = find_peaks(mag, height=mag.mean() + mag.std(), distance=3)
    window_duration = (window_data['timestamp'].max() - window_data['timestamp'].min()).total_seconds()
    features['step_frequency'] = len(peaks) / window_duration if window_duration > 0 else 0
    
    # Convert to numerical array (exclude subject)
    feature_values = [v for k, v in features.items()]
    return np.array(feature_values)

# Prepare sequences
def prepare_sequences(df, max_seq_len=200):
    sequences = []
    labels = []
    subjects = []
    max_len = 0
    for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
        windows, window_labels, window_subjects = generate_windows(group)
        if windows:
            sequence_features = [extract_features(window) for window in windows]
            seq_len = len(sequence_features)
            if seq_len > max_seq_len:
                sequence_features = sequence_features[:max_seq_len]
                seq_len = max_seq_len
            max_len = max(max_len, seq_len)
            sequences.append(np.array(sequence_features))
            labels.append(window_labels[0])
            subjects.append(window_subjects[0])
    print(f"Maximum sequence length: {max_len}")
    return sequences, labels, subjects, max_len

# Custom Dataset
class ActivityDataset(Dataset):
    def __init__(self, sequences, labels, subjects, subject_encoder):
        self.sequences = sequences
        self.labels = labels
        self.subjects = [subject_encoder.get(subj, len(subject_encoder)) for subj in subjects]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.subjects[idx]

def collate_fn(batch):
    sequences, labels, subjects = zip(*batch)
    max_len = max(seq.shape[0] for seq in sequences)
    padded_sequences = []
    masks = []
    for seq in sequences:
        pad_width = ((0, max_len - seq.shape[0]), (0, 0))
        padded_seq = np.pad(seq, pad_width, mode='constant', constant_values=0)
        padded_sequences.append(padded_seq)
        mask = np.ones(seq.shape[0], dtype=bool)
        mask = np.pad(mask, (0, max_len - seq.shape[0]), mode='constant', constant_values=False)
        masks.append(mask)
    return (torch.tensor(padded_sequences, dtype=torch.float32),
            torch.tensor(masks, dtype=torch.bool),
            torch.tensor(labels, dtype=torch.long),
            torch.tensor(subjects, dtype=torch.long))

# Transformer Model
class ActivityTransformer(nn.Module):
    def __init__(self, num_features, num_classes, num_subjects, max_seq_len, d_model=128, num_heads=4, num_layers=2, dropout=0.1, embedding_dim=32):
        super(ActivityTransformer, self).__init__()
        self.d_model = d_model
        self.subject_embedding = nn.Embedding(num_subjects + 1, embedding_dim)
        self.input_proj = nn.Linear(num_features + embedding_dim, d_model)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.pos_encoder = nn.Parameter(torch.zeros(1, max_seq_len + 1, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads, dim_feedforward=512, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.classifier = nn.Linear(d_model, num_classes)
    
    def forward(self, x, mask, subject_ids):
        batch_size, seq_len, num_features = x.shape
        subj_emb = self.subject_embedding(subject_ids).unsqueeze(1).expand(-1, seq_len, -1)
        x = torch.cat((x, subj_emb), dim=-1)
        x = self.input_proj(x)
        cls_token = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        pos_encoding = self.pos_encoder[:, :seq_len + 1, :]
        x += pos_encoding
        cls_mask = torch.ones(batch_size, 1, dtype=torch.bool, device=x.device)
        full_mask = torch.cat((cls_mask, mask), dim=1)
        transformer_mask = ~full_mask
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x, src_key_padding_mask=transformer_mask)
        x = x.permute(1, 0, 2)
        cls_output = x[:, 0, :]
        return self.classifier(cls_output)

# Training and Evaluation Functions
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        sequences, masks, labels, subjects = [b.to(device) for b in batch]
        optimizer.zero_grad()
        logits = model(sequences, masks, subjects)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            sequences, masks, labels, subjects = [b.to(device) for b in batch]
            logits = model(sequences, masks, subjects)
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    return accuracy_score(true, preds), f1_score(true, preds, average='weighted')



    # Prepare data
sequences, labels, subjects, max_seq_len = prepare_sequences(df)
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
subject_encoder = {subj: idx for idx, subj in enumerate(sorted(set(subjects)))}

# Split for a single fold
train_idx = [i for i, s in enumerate(subjects) if s not in 'U3']
val_idx = [i for i, s in enumerate(subjects) if s == 'U3']
train_data = ([sequences[i] for i in train_idx], labels_encoded[train_idx], [subjects[i] for i in train_idx])
val_data = ([sequences[i] for i in val_idx], labels_encoded[val_idx], [subjects[i] for i in val_idx])

# Datasets and Loaders
train_dataset = ActivityDataset(*train_data, subject_encoder)
val_dataset = ActivityDataset(*val_data, subject_encoder)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Model Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = sequences[0].shape[1]
model = ActivityTransformer(
    num_features=num_features,
    num_classes=len(le.classes_),
    num_subjects=len(subject_encoder),
    max_seq_len=max_seq_len
).to(device)
class_weights = torch.tensor([1.0 / Counter(labels_encoded)[i] for i in range(len(le.classes_))], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Training Loop
for epoch in range(30):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_acc, val_f1 = evaluate(model, val_loader, device)
    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val F1={val_f1:.4f}")
    scheduler.step(train_loss)

In [31]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing: Normalize per subject
def normalize_per_subject(df):
    scaler = StandardScaler()
    for subject in df['subject'].unique():
        mask = df['subject'] == subject
        df.loc[mask, ['x', 'y', 'z']] = scaler.fit_transform(df.loc[mask, ['x', 'y', 'z']])
    return df

# Data Augmentation: Add jitter
def augment_data(window_data):
    noise = np.random.normal(0, 0.01, window_data[['x', 'y', 'z']].shape)
    window_data[['x', 'y', 'z']] += noise
    return window_data

# Generate Windows
def generate_windows(group, window_size=pd.Timedelta(seconds=1), step_size=pd.Timedelta(seconds=0.5)):
    group = group.sort_values('timestamp')
    start_time = group['timestamp'].min()
    end_time = group['timestamp'].max()
    windows = []
    labels = []
    subjects = []
    current_start = start_time
    
    while current_start + window_size <= end_time:
        window_end = current_start + window_size
        window_data = group[(group['timestamp'] >= current_start) & (group['timestamp'] < window_end)]
        if not window_data.empty:
            # Apply augmentation with 50% probability
            if np.random.rand() > 0.5:
                window_data = augment_data(window_data.copy())
            windows.append(window_data)
            labels.append(group['activity_type'].iloc[0])
            subjects.append(group['subject'].iloc[0])
        current_start += step_size
    
    return windows, labels, subjects

# Feature Extraction
def extract_features(window_data):
    features = {}
    
    # Time-Domain Features
    for axis in ['x', 'y', 'z']:
        data = window_data[axis]
        features[f'{axis}_mean'] = data.mean()
        features[f'{axis}_std'] = data.std()
        features[f'{axis}_skew'] = skew(data)
        features[f'{axis}_kurt'] = kurtosis(data)
        features[f'{axis}_min'] = data.min()
        features[f'{axis}_max'] = data.max()
        features[f'{axis}_range'] = data.max() - data.min()
    
    # Magnitude Features
    mag = np.sqrt(window_data['x']**2 + window_data['y']**2 + window_data['z']**2)
    features['mag_mean'] = mag.mean()
    features['mag_std'] = mag.std()
    
    # Cross-Axis Correlation
    features['xy_corr'] = window_data['x'].corr(window_data['y'])
    features['xz_corr'] = window_data['x'].corr(window_data['z'])
    features['yz_corr'] = window_data['y'].corr(window_data['z'])
    
    # Frequency-Domain Features
    for axis in ['x', 'y', 'z']:
        fft_vals = np.abs(np.fft.fft(window_data[axis]))[:len(window_data[axis])//2]
        freqs = np.fft.fftfreq(len(window_data[axis]), d=0.135)[:len(window_data[axis])//2]
        for low, high in [(0, 1), (1, 2), (2, 3), (3, 3.7)]:
            mask = (freqs >= low) & (freqs < high)
            features[f'{axis}_power_{low}-{high}'] = np.sum(fft_vals[mask]**2) if mask.any() else 0
    
    # Step Frequency
    peaks, _ = find_peaks(mag, height=mag.mean() + mag.std(), distance=3)
    window_duration = (window_data['timestamp'].max() - window_data['timestamp'].min()).total_seconds()
    features['step_frequency'] = len(peaks) / window_duration if window_duration > 0 else 0
    
    # Convert to numerical array
    feature_values = [v if not np.isnan(v) else 0 for k, v in features.items()]
    return np.array(feature_values)

# Prepare Sequences
def prepare_sequences(df, max_seq_len=50):
    sequences = []
    labels = []
    subjects = []
    max_len = 0
    for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
        windows, window_labels, window_subjects = generate_windows(group)
        if windows:
            sequence_features = [extract_features(window) for window in windows]
            seq_len = len(sequence_features)
            if seq_len > max_seq_len:
                sequence_features = sequence_features[:max_seq_len]
                seq_len = max_seq_len
            max_len = max(max_len, seq_len)
            sequences.append(np.array(sequence_features))
            labels.append(window_labels[0])
            subjects.append(window_subjects[0])
    print(f"Maximum sequence length: {max_len}")
    return sequences, labels, subjects, max_len

# Custom Dataset
class ActivityDataset(Dataset):
    def __init__(self, sequences, labels, subjects, subject_encoder):
        self.sequences = sequences
        self.labels = labels
        self.subjects = [subject_encoder.get(subj, len(subject_encoder)) for subj in subjects]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.subjects[idx]

def collate_fn(batch):
    sequences, labels, subjects = zip(*batch)
    max_len = max(seq.shape[0] for seq in sequences)
    padded_sequences = []
    masks = []
    for seq in sequences:
        pad_width = ((0, max_len - seq.shape[0]), (0, 0))
        padded_seq = np.pad(seq, pad_width, mode='constant', constant_values=0)
        padded_sequences.append(padded_seq)
        mask = np.ones(seq.shape[0], dtype=bool)
        mask = np.pad(mask, (0, max_len - seq.shape[0]), mode='constant', constant_values=False)
        masks.append(mask)
    return (torch.tensor(padded_sequences, dtype=torch.float32),
            torch.tensor(masks, dtype=torch.bool),
            torch.tensor(labels, dtype=torch.long),
            torch.tensor(subjects, dtype=torch.long))

# Transformer Model
class ActivityTransformer(nn.Module):
    def __init__(self, num_features, num_classes, num_subjects, max_seq_len, d_model=64, num_heads=4, num_layers=1, dropout=0.3, embedding_dim=16):
        super(ActivityTransformer, self).__init__()
        self.d_model = d_model
        self.subject_embedding = nn.Embedding(num_subjects + 1, embedding_dim)
        self.input_proj = nn.Linear(num_features + embedding_dim, d_model)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.pos_encoder = nn.Parameter(torch.zeros(1, max_seq_len + 1, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads, dim_feedforward=256, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.classifier = nn.Linear(d_model, num_classes)
    
    def forward(self, x, mask, subject_ids):
        batch_size, seq_len, num_features = x.shape
        subj_emb = self.subject_embedding(subject_ids).unsqueeze(1).expand(-1, seq_len, -1)
        x = torch.cat((x, subj_emb), dim=-1)
        x = self.input_proj(x)
        cls_token = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        pos_encoding = self.pos_encoder[:, :seq_len + 1, :]
        x += pos_encoding
        cls_mask = torch.ones(batch_size, 1, dtype=torch.bool, device=x.device)
        full_mask = torch.cat((cls_mask, mask), dim=1)
        transformer_mask = ~full_mask
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x, src_key_padding_mask=transformer_mask)
        x = x.permute(1, 0, 2)
        cls_output = x[:, 0, :]
        return self.classifier(cls_output)

# Training and Evaluation Functions
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        sequences, masks, labels, subjects = [b.to(device) for b in batch]
        optimizer.zero_grad()
        logits = model(sequences, masks, subjects)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, device, le):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            sequences, masks, labels, subjects = [b.to(device) for b in batch]
            logits = model(sequences, masks, subjects)
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds, average='weighted')
    # Per-class F1
    per_class_f1 = f1_score(true, preds, average=None)
    print("Per-class F1-scores:", dict(zip(le.classes_, per_class_f1)))
    # Confusion Matrix
    cm = confusion_matrix(true, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix.png')
    plt.close()
    return acc, f1

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
    
    def __call__(self, val_f1, model):
        score = val_f1
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0


# Preprocess
df = normalize_per_subject(df)
sequences, labels, subjects, max_seq_len = prepare_sequences(df)
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
subject_encoder = {subj: idx for idx, subj in enumerate(sorted(set(subjects)))}

# Split for a single fold
train_idx = [i for i, s in enumerate(subjects) if s != 'U3']
val_idx = [i for i, s in enumerate(subjects) if s == 'U3']
train_data = ([sequences[i] for i in train_idx], labels_encoded[train_idx], [subjects[i] for i in train_idx])
val_data = ([sequences[i] for i in val_idx], labels_encoded[val_idx], [subjects[i] for i in val_idx])

# Datasets and Loaders
train_dataset = ActivityDataset(*train_data, subject_encoder)
val_dataset = ActivityDataset(*val_data, subject_encoder)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

# Model Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = sequences[0].shape[1]  # Should be 24: (7 time + 4 freq) * 3 axes + 2 mag + 3 corr + 1 step
model = ActivityTransformer(
    num_features=num_features,
    num_classes=len(le.classes_),
    num_subjects=len(subject_encoder),
    max_seq_len=max_seq_len,
    d_model=64,
    num_heads=4,
    num_layers=1,
    dropout=0.3,
    embedding_dim=16
).to(device)
class_weights = torch.tensor([1.0 / Counter(labels_encoded)[i] for i in range(len(le.classes_))], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
early_stopping = EarlyStopping(patience=5)

# Training Loop
for epoch in range(50):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_acc, val_f1 = evaluate(model, val_loader, device, le)
    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val F1={val_f1:.4f}")
    scheduler.step(train_loss)
    # early_stopping(val_f1, model)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

Maximum sequence length: 50
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 0.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.0, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.3333333333333333, '7 Cool down - sitting/relax': 0.0}


c:\Users\rohan\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\rohan\AppData\Local\Temp\ipykernel_49192\2909796749.py:141: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  torch.tensor(masks, dtype=torch.bool),


Epoch 1: Loss=2.5765, Val Acc=0.1000, Val F1=0.0333
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 1.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 0.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.0, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.6666666666666666, '7 Cool down - sitting/relax': 0.0}


C:\Users\rohan\AppData\Local\Temp\ipykernel_49192\2909796749.py:141: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  torch.tensor(masks, dtype=torch.bool),


Epoch 2: Loss=2.3322, Val Acc=0.2000, Val F1=0.1667
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 1.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.0, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.0, '7 Cool down - sitting/relax': 0.0, '8 Walk (LEFT --> Right --> Left)': 0.0}


...

Epoch 48: Loss=1.0741, Val Acc=0.3000, Val F1=0.2500
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 1.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.0, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.5, '7 Cool down - sitting/relax': 0.0}


C:\Users\rohan\AppData\Local\Temp\ipykernel_49192\2909796749.py:141: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  torch.tensor(masks, dtype=torch.bool),


Epoch 49: Loss=1.0632, Val Acc=0.3000, Val F1=0.2500
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 1.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.0, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.5, '7 Cool down - sitting/relax': 0.0}


C:\Users\rohan\AppData\Local\Temp\ipykernel_49192\2909796749.py:141: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  torch.tensor(masks, dtype=torch.bool),


Epoch 50: Loss=1.0062, Val Acc=0.3000, Val F1=0.2500
